# Reading in databases and LCI 

This notebook set up the project and import and create the necessary databases. 

In [1]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [1]:
from matplotlib import pyplot as plt
from brightway2 import * 
import numpy as np
import seaborn as sns
import pandas as pd

projects.set_current("scrappage programs")

Importing the biosphere matrix and ecoinvent:

In [5]:
bw2setup()

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.00 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/08/2019 16:54:00
  Finished: 07/08/2019 16:54:00
  Total time elapsed: 00:00:00
  CPU %: 88.00
  Memory %: 1.82
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 2.04 seconds
Wrote 850 LCIA methods with 220699 characterization factors
Creating core data migrations



In [3]:
fpei33 = "C:/Users/nistad_a/datasets"

if 'ecoinvent 3.5 cutoff' in databases:
    print("Database has already been imported")
else:
    ei33 = SingleOutputEcospold2Importer(fpei33, 'ecoinvent 3.5 cutoff')
    ei33.apply_strategies()
    ei33.statistics()

Database has already been imported


In [13]:
ei33.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:18


Title: Writing activities to SQLite3 database:
  Started: 07/08/2019 17:10:30
  Finished: 07/08/2019 17:12:48
  Total time elapsed: 00:02:18
  CPU %: 74.40
  Memory %: 18.11
Created database: ecoinvent 3.5 cutoff


Brightway2 SQLiteBackend: ecoinvent 3.5 cutoff

In [2]:
databases

Databases dictionary with 12 object(s):
	Additional datasets
	Current cars
	biosphere3
	ecoinvent 3.5 cutoff
	euro 0
	euro 1
	euro 2
	euro 3
	euro 4
	euro 5
	euro 6
	pre euro 0

Read in inventory and create databases for all cars: 

In [3]:
filepath = "./data/inventory"
filepath+="/lci-battery-glider"
filepath+=".xlsx"

imp = ExcelImporter(filepath)
imp.apply_strategies()
imp.match_database(fields=('name', 'unit', 'location'))

imp.match_database("ecoinvent 3.5 cutoff", fields=('name', 'unit', 'location'))
imp.statistics()
imp.write_database()

Extracted 2 worksheets in 0.17 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.53 seconds


In [4]:
def lci_euro_to_db(euro_class):
    filepath = "./data/inventory/lci-"
    filepath+=euro_class
    filepath+=".xlsx"
    print(filepath)

    imp = ExcelImporter(filepath)
    imp.apply_strategies()
    imp.match_database(fields=('name', 'unit', 'location'))

    imp.match_database("Additional datasets", fields=('name', 'unit', 'location'))
    #imp.match_database("Additional datasets region", fields=('name', 'unit', 'location'))
    imp.match_database("ecoinvent 3.5 cutoff", fields=('name', 'unit', 'location'))
    imp.statistics()

    if imp.statistics()[-1] == 0: 
        imp.write_database()

In [5]:
euro_classes_to_db = ["preeuro0", "euro0", "euro1", "euro2", "euro3", "euro4", "euro5", "euro6", "currentcars"]
for euro_class in euro_classes_to_db: 
    lci_euro_to_db(euro_class)

./data/inventory/lci-euro0.xlsx
Extracted 1 worksheets in 0.08 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.40 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
14 datasets


Writing activities to SQLite3 database:
0% [##############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 07/09/2019 09:29:29
  Finished: 07/09/2019 09:29:29
  Total time elapsed: 00:00:00
  CPU %: 81.80
  Memory %: 2.12
Created database: euro 0


In [6]:
databases

Databases dictionary with 12 object(s):
	Additional datasets
	Current cars
	biosphere3
	ecoinvent 3.5 cutoff
	euro 0
	euro 1
	euro 2
	euro 3
	euro 4
	euro 5
	euro 6
	pre euro 0

Know that something is wrong with the biogas in electricity production in the current version of ecoinvent - so change this: 

In [11]:
act = [x for x in Database("ecoinvent 3.5 cutoff") if "anaerobic digestion of manure" in x['name']]

In [24]:
ammonia = [exc for exc in act[1].biosphere() if 'Ammonia' in exc['name']][0]
print(ammonia['amount'])
ammonia['amount'] = 0.0009 
ammonia.save()


0.0009
